<a href="https://colab.research.google.com/github/ohsono/stats414/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# This a sandbox where you can test your code block without messing around the master notebook**

Feel free to write your code or delete but please don't delete the top code block

In [ ]:
#### This is an optional code block                                  ####
#### please execute it once or none if you have the dataset uploaded ####

import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("xiaojiu1414/digix-global-ai-challenge")
# Print the current working directory
train_data=path+"/train/"
print("Current working directory: {0}".format(os.getcwd()))
# Change the current working directory
os.chdir(train_data)
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

100%|██████████| 879M/879M [00:06<00:00, 138MB/s]

Extracting files...


Current working directory: /content
Current working directory: /root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_score,recall_score,f1_score
from IPython.display import display
from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
if path is None:
  %cd /content/sample_data/
  !ls

In [ ]:
train_ads = pd.read_csv('train_data_ads.csv')
train_ads.head()#.T

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,35,312,2117,7,14339,19319,6,1846,4,46,1,162,47,14,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,35,312,2117,7,11923,21714,8,1060,4,37,1,344,20,13,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,35,312,2117,7,11923,21714,8,1060,4,37,1,344,20,13,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0,100005,3,2,16,147,2,32,6,35,312,2117,7,28665,20163,8,1542,4,47,1,152,49,17,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0,100005,3,2,16,147,2,32,6,35,312,2117,7,34041,13608,8,1036,4,44,1,312,43,18,10.0,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15


In [ ]:
train_ads['adcl1'] = train_ads['ad_click_list_v001'].str.count('\^') + 1
train_ads['adcl2'] = train_ads['ad_click_list_v002'].str.count('\^') + 1
train_ads['adcl3'] = train_ads['ad_click_list_v003'].str.count('\^') + 1

total_clicks = train_ads['adcl1'].sum()+train_ads['adcl2'].sum()++train_ads['adcl3'].sum()
print(f"Total clicks: {total_clicks}")

Total clicks: 93995886


In [ ]:
train_ads['adco1'] = train_ads['ad_close_list_v001'].str.count('\^') + 1
train_ads['adco2'] = train_ads['ad_close_list_v002'].str.count('\^') + 1
train_ads['adco3'] = train_ads['ad_close_list_v003'].str.count('\^') + 1

total_close = train_ads['adco1'].sum()+train_ads['adco2'].sum()++train_ads['adco3'].sum()
print(f"Total clicks: {total_close}")

Total clicks: 24937873


In [ ]:
 np.random.seed(42)

labels = train_ads['label']
unique_labels = labels.unique()

# sample 10% from 0 and sample 10% from 1,
# extract only 10% sampling
sampled_df = pd.DataFrame()
for label in unique_labels:
  label_data = train_ads[labels==label]
  label_sample = label_data.sample(
      n=int(len(label_data) * 0.1),
      random_state=42
  )
  sampled_df = pd.concat([sampled_df, label_sample])

sampled_df.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle,adcl1,adcl2,adcl3,adco1,adco2,adco3
5640147,229871,0,102874,6,4,33,319,3,27,2,11,346,2032,7,34382,11752,10,2066,4,16,1,114,43,18,10.0,12493^30157^36061^13723^21950,1068^2066^2035^1633^1220,190^114^168^162^280,24107,1218,173,202206080318,53^219^17^112^86,3,17,5,5,5,1,1,1
6794328,975220,0,121845,6,4,33,319,3,27,2,11,278,1555,7,16598,10206,5,1296,3,22,1,213,18,23,0.0,16151^11393^32551^13227^24055,1112^1633^1913^1650^1014,162^257^312^367,24107,1218,173,202206091230,171^207^218^39^168,9,17,5,5,4,1,1,1
1232570,1116390,0,108413,3,4,46,181,2,30,3,13,272,1777,6,31996,10724,10,1403,4,16,1,174,16,18,10.0,16520^31433^20329^14250^31751,1650^1558^1380^1173^1830,162^298^210,24107,1218,173,202206040858,56^195^168^86^78,8,17,5,5,3,1,1,1
775136,132715,0,223178,8,3,27,162,5,31,3,20,332,2117,7,29564,17856,4,1236,3,31,1,213,18,23,0.0,23535^14747^34001^24692^34992,1216^1245^1269^1340^1369,162^306^114^298,24107,1218,173,202206030919,98^116^98^98^105,8,17,5,5,4,1,1,1
3363238,126046,0,267374,2,2,33,319,3,34,7,20,224,2117,4,34125,12938,8,1515,5,37,1,162,47,14,10.0,22827^12443^32776^30393^17781,1314^1223^1690^2011^1344,246^162^150^111^142,24107,1218,173,202206050924,78^98^195^0^0,0,16,5,5,5,1,1,1


In [ ]:
print("original_df:{}".format(len(train_ads)))
print("sampled_df:{}".format(len(sampled_df)))
#print(len(sampled_df))
sampled_df.drop(['ad_click_list_v001','ad_click_list_v002','ad_click_list_v003','ad_close_list_v001','ad_close_list_v002','ad_close_list_v003','u_newsCatInterestsST'],axis=1,inplace=True)
sampled_df.head()

original_df:7675517
sampled_df:767551


,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,emui_dev,device_name,device_size,net_type,task_id,adv_id,creat_type_cd,adv_prim_id,inter_type_cd,slot_id,site_id,spread_app_id,hispace_app_tags,app_second_class,app_score,pt_d,u_refreshTimes,u_feedLifeCycle,adcl1,adcl2,adcl3,adco1,adco2,adco3
5640147,229871,0,102874,6,4,33,319,3,27,2,11,346,2032,7,34382,11752,10,2066,4,16,1,114,43,18,10.0,202206080318,3,17,5,5,5,1,1,1
6794328,975220,0,121845,6,4,33,319,3,27,2,11,278,1555,7,16598,10206,5,1296,3,22,1,213,18,23,0.0,202206091230,9,17,5,5,4,1,1,1
1232570,1116390,0,108413,3,4,46,181,2,30,3,13,272,1777,6,31996,10724,10,1403,4,16,1,174,16,18,10.0,202206040858,8,17,5,5,3,1,1,1
775136,132715,0,223178,8,3,27,162,5,31,3,20,332,2117,7,29564,17856,4,1236,3,31,1,213,18,23,0.0,202206030919,8,17,5,5,4,1,1,1
3363238,126046,0,267374,2,2,33,319,3,34,7,20,224,2117,4,34125,12938,8,1515,5,37,1,162,47,14,10.0,202206050924,0,16,5,5,5,1,1,1


In [ ]:
import gc
#delete unused memory for train_ads
def get_memory_usage(df: pd.DataFrame) -> str:
    """Get memory usage of DataFrame in MB"""
    memory_mb = df.memory_usage(deep=True).sum() / 1024 / 1024
    return f"{memory_mb:.2f} MB"
get_memory_usage(train_ads)
get_memory_usage(sampled_df)
del train_ads
gc.collect()

8

In [ ]:
!pip install xgboost

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score

pca_analyzer=PCA(n_components=0.95)
transformed_data = pca_analyzer.fit_transform(sampled_df)
transformed_df = pd.DataFrame(transformed_data)

# Access variance explained by each component
variance_df = pca_analyzer.explained_variance_ratio_
# PCA doesn't directly provide feature importance, but you can get component loadings
feature_importance = pca_analyzer.components_

# Instead of get_optimal_components, find the number of components explaining 95% variance
cumulative_variance = np.cumsum(variance_df)
optimal_n_components = np.argmax(cumulative_variance >= 0.95) + 1

# Get the names of the components (if available)
component_names = [f"PC{i+1}" for i in range(transformed_data.shape[1])]

# Select important features based on optimal components
important_features = transformed_data[:, :optimal_n_components]

sampled_labels = sampled_df['label']
X_train, X_test, y_train, y_test = train_test_split(
    transformed_data, sampled_labels, test_size=0.2, random_state=42
)

results = {}
models = {
    'logistic': LogisticRegression(random_state=42),
    #'random_forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'xgboost': XGBClassifier(random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = model.predict(X_test)

    # Calculate metrics
    results[name] = {
        'auc_roc': roc_auc_score(y_test, y_pred_proba),
        'f1_score': f1_score(y_test, y_pred),
        # Use component names instead of self.component_names (which is undefined)
        'feature_importance': dict(zip(component_names,
                                    abs(model.coef_[0]) if name == 'logistic'
                                    else model.feature_importances_))
    }

In [ ]:
transformed_df.head()
print("\nVariance:{}".format(variance_df))
print("\nfeature_importance:{}".format(feature_importance))


Variance:[0.96791365]

feature_importance:[[ 9.99983589e-01 -2.63914332e-10  1.54847774e-03  3.03229197e-09
   3.73888231e-08  4.58891860e-08  2.48321037e-06 -3.33851715e-08
  -1.40799811e-07 -5.13271835e-08 -2.97535563e-07 -1.99932907e-06
   1.08781176e-05  3.41434342e-08  2.93520786e-05  9.82136216e-06
   3.08605723e-08 -4.34787777e-06 -1.51914685e-08 -3.39993039e-07
   1.11946042e-17  1.05758884e-06  6.61333463e-08 -5.50479995e-08
   5.10139467e-08 -5.51562342e-03  8.73004398e-08  7.26346792e-09
   1.98866103e-08  2.47920082e-08  3.85193993e-08 -1.34370293e-09
  -7.57815322e-10  2.05685114e-09]]
